In [71]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
%matplotlib inline

In [72]:
train_file = 'training_1.pickle'
with open(train_file, 'rb') as f1:
	training_data = pickle.load(f1)

In [3]:
training_data.keys()

['descriptors', 'name', 'activity']

In [4]:
training_data['name'][0:5]

array(['ACT1_M_80', 'ACT1_M_189', 'ACT1_M_190', 'ACT1_M_402', 'ACT1_M_659'], 
      dtype='|S13')

In [73]:
name = training_data['name']

In [74]:
activity = training_data['activity']

In [13]:
activity.shape

(37241,)

In [75]:
descriptors = training_data['descriptors']

In [8]:
activity[0]

6.0179

In [14]:
activity[0:5, ]

array([ 6.0179,  4.3003,  5.2697,  6.1797,  4.3003])

In [15]:
descriptors[0:3,]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [9]:
activity[0:5]

array([ 6.0179,  4.3003,  5.2697,  6.1797,  4.3003])

In [10]:
descriptors[0][0:10]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes

In [76]:
import tensorflow as tf
from scipy import stats

In [79]:
# simple neural network 
batch_size = 1000

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, 9491))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, ))
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([9491, 1024]))
  biases_1 = tf.Variable(tf.zeros([1024]))
  weights_2 = tf.Variable(tf.truncated_normal([1024, 1]))
  biases_2 = tf.Variable(tf.zeros([1]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  output = tf.add(tf.matmul(hidden1, weights_2), biases_2)
  print (output.get_shape())
  print (tf_train_labels.get_shape())
  cost = tf.reduce_sum(tf.pow(output-tf_train_labels, 2))

  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(cost)
  print (type(cost))
  

(1000, 1)
(1000,)
<class 'tensorflow.python.framework.ops.Tensor'>


In [80]:
num_steps = 5001
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (activity.shape[0] - batch_size)
        batch_data = descriptors[offset:(offset+batch_size), ]
        batch_labels = activity[offset:(offset+batch_size), ]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels: batch_labels}
        cost1, predictions = session.run([cost, output], feed_dict = feed_dict)
        
        if (step % 500 == 0):
        
          print("Minibatch loss at step %d: %f" % (step, cost1))
    

Initialized
Minibatch loss at step 0: 10379574902784.000000
Minibatch loss at step 500: 13211396997120.000000
Minibatch loss at step 1000: 13004832768000.000000
Minibatch loss at step 1500: 13517856964608.000000
Minibatch loss at step 2000: 14304366559232.000000
Minibatch loss at step 2500: 12570084769792.000000
Minibatch loss at step 3000: 12910633943040.000000
Minibatch loss at step 3500: 13887982272512.000000
Minibatch loss at step 4000: 13175148773376.000000
Minibatch loss at step 4500: 13244400926720.000000
Minibatch loss at step 5000: 13409680621568.000000
